# 👨‍💼👨‍🚀👨‍💻 RAG with File Retrieval

This notebook is restricted to OpenAI API only!

### Initialize the kernel

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.18.0-rc"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.18.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.18.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.18.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.OpenAI, 1.18.0-alpha"

#!import ../../Secrets.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.Agents.OpenAI;
using Microsoft.SemanticKernel.ChatCompletion;

var kernel = Kernel.CreateBuilder()
    .AddOpenAIChatCompletion(
        modelId: Secrets.OpenAIModel,
        apiKey: Secrets.OpenAIApiKey)
    .Build();

### Let's create a chat group for agents and add a goal to the chat.

In [ ]:
#pragma warning disable SKEXP0110

AgentGroupChat chat = new();

var goal = """
    Jack Sparrow makes a bad joke about Don Quijote's new favorite drink and his new horse.
    """;

chat.AddChatMessage(new ChatMessageContent(AuthorRole.User, goal));


### Read _details.txt_ file and upload it to the model

In [ ]:
#pragma warning disable SKEXP0010
#pragma warning disable SKEXP0001

var fileService = new OpenAIFileService(Secrets.OpenAIApiKey);
var bytes = await System.IO.File.ReadAllBytesAsync("../../Resources/Data/details.txt");
var binaryContent = new BinaryContent(bytes, mimeType: "text/plain");
var executionSettings = new OpenAIFileUploadExecutionSettings("../../Resources/Data/details.txt", OpenAIFilePurpose.Assistants);
var uploadFile = await fileService.UploadContentAsync(
    binaryContent,
    executionSettings
);

### Create Jack Sparrow agent with file retrieval capabilities (agents are using the out-of-the-box RAG feature of the OpenAI assistants)

In [ ]:
#pragma warning disable SKEXP0110

using System.ClientModel;

var agent = await OpenAIAssistantAgent.CreateAsync(
    kernel: kernel,
    clientProvider: OpenAIClientProvider.ForOpenAI(new ApiKeyCredential(Secrets.OpenAIApiKey!)),
    definition: new OpenAIAssistantDefinition(Secrets.OpenAIModel)
    {
        Instructions = """
            You are Jack Sparrow talking in Jack Sparrow style.
            Evaluate the context and reply providing exactly one meaningful dialog line.
            The dialog line must be only one sentence of maximum 50 words.
            """,
        Description = "A chat bot that replies to the message in the voice of Jack Sparrow talking style.",
        Name = "JackSparrow",
        Id = "JackSparrow_01",
        EnableFileSearch = true,
        CodeInterpreterFileIds = [uploadFile.Id]
    });

### ⚡ The chat is invoking Jack Sparrow agent

In [ ]:
#pragma warning disable SKEXP0001
#pragma warning disable SKEXP0110

Console.WriteLine($"{AuthorRole.User} > {goal}.");

await foreach (var content in chat.InvokeAsync(agent))
{
    Console.WriteLine($"{content.Role} [{content.AuthorName}] > {content.Content}");
}

### Notice that the agent is fetching additional context from the external file _details.txt_